<a href="https://colab.research.google.com/github/MajiroZ/for_git_study/blob/master/Seq2Seq.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##【問題1】機械翻訳の実行とコードリーディング

指定されたURLがpage not foundになってしまったため、検索して出てきたコードを使用した。

In [ ]:
"""
Title: Character-level recurrent sequence-to-sequence model
Author: [fchollet](https://twitter.com/fchollet)
Date created: 2017/09/29
Last modified: 2023/11/22
Description: Character-level recurrent sequence-to-sequence model.
Accelerator: GPU
"""

"""
## Introduction

This example demonstrates how to implement a basic character-level
recurrent sequence-to-sequence model. We apply it to translating
short English sentences into short French sentences,
character-by-character. Note that it is fairly unusual to
do character-level machine translation, as word-level
models are more common in this domain.

**Summary of the algorithm**

- We start with input sequences from a domain (e.g. English sentences)
    and corresponding target sequences from another domain
    (e.g. French sentences).
- An encoder LSTM turns input sequences to 2 state vectors
    (we keep the last LSTM state and discard the outputs).
- A decoder LSTM is trained to turn the target sequences into
    the same sequence but offset by one timestep in the future,
    a training process called "teacher forcing" in this context.
    It uses as initial state the state vectors from the encoder.
    Effectively, the decoder learns to generate `targets[t+1...]`
    given `targets[...t]`, conditioned on the input sequence.
- In inference mode, when we want to decode unknown input sequences, we:
    - Encode the input sequence into state vectors
    - Start with a target sequence of size 1
        (just the start-of-sequence character)
    - Feed the state vectors and 1-char target sequence
        to the decoder to produce predictions for the next character
    - Sample the next character using these predictions
        (we simply use argmax).
    - Append the sampled character to the target sequence
    - Repeat until we generate the end-of-sequence character or we
        hit the character limit.
"""

"""
## Setup
"""

import numpy as np
import keras
import os
from pathlib import Path

"""
## Download the data
"""

# url = "http://www.manythings.org/anki/fra-eng.zip"
# fpath = os.path.basename(url) # Extract the filename from the URL
# if not os.path.exists(fpath):
#   wget.download(url)
fpath = "/fra.txt"

dirpath = Path(fpath).parent.absolute()
os.system(f"unzip -q {fpath} -d {dirpath}")

"""
## Configuration
"""

batch_size = 128  # Batch size for training.
epochs = 50  # Number of epochs to train for.
latent_dim = 256  # Latent dimensionality of the encoding space.
num_samples = 5000  # Number of samples to train on.
# Path to the data txt file on disk.
data_path = os.path.join(dirpath, "fra.txt")

"""
## Prepare the data
"""

# Vectorize the data.
input_texts = []
target_texts = []
input_characters = set()
target_characters = set()
with open(data_path, "r", encoding="utf-8") as f:
    lines = f.read().split("\n")
for line in lines[: min(num_samples, len(lines) - 1)]:
    input_text, target_text, _ = line.split("\t")
    # We use "tab" as the "start sequence" character
    # for the targets, and "\n" as "end sequence" character.
    target_text = "\t" + target_text + "\n"
    input_texts.append(input_text)
    target_texts.append(target_text)
    for char in input_text:
        if char not in input_characters:
            input_characters.add(char)
    for char in target_text:
        if char not in target_characters:
            target_characters.add(char)

input_characters = sorted(list(input_characters))
target_characters = sorted(list(target_characters))
num_encoder_tokens = len(input_characters)
num_decoder_tokens = len(target_characters)
max_encoder_seq_length = max([len(txt) for txt in input_texts])
max_decoder_seq_length = max([len(txt) for txt in target_texts])

print("Number of samples:", len(input_texts))
print("Number of unique input tokens:", num_encoder_tokens)
print("Number of unique output tokens:", num_decoder_tokens)
print("Max sequence length for inputs:", max_encoder_seq_length)
print("Max sequence length for outputs:", max_decoder_seq_length)

input_token_index = dict([(char, i) for i, char in enumerate(input_characters)])
target_token_index = dict([(char, i) for i, char in enumerate(target_characters)])

encoder_input_data = np.zeros(
    (len(input_texts), max_encoder_seq_length, num_encoder_tokens),
    dtype="float32",
)
decoder_input_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens),
    dtype="float32",
)
decoder_target_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens),
    dtype="float32",
)

for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
    for t, char in enumerate(input_text):
        encoder_input_data[i, t, input_token_index[char]] = 1.0
    encoder_input_data[i, t + 1 :, input_token_index[" "]] = 1.0
    for t, char in enumerate(target_text):
        # decoder_target_data is ahead of decoder_input_data by one timestep
        decoder_input_data[i, t, target_token_index[char]] = 1.0
        if t > 0:
            # decoder_target_data will be ahead by one timestep
            # and will not include the start character.
            decoder_target_data[i, t - 1, target_token_index[char]] = 1.0
    decoder_input_data[i, t + 1 :, target_token_index[" "]] = 1.0
    decoder_target_data[i, t:, target_token_index[" "]] = 1.0

"""
## Build the model
"""

# Define an input sequence and process it.
encoder_inputs = keras.Input(shape=(None, num_encoder_tokens))
encoder = keras.layers.LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)

# We discard `encoder_outputs` and only keep the states.
encoder_states = [state_h, state_c]

# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = keras.Input(shape=(None, num_decoder_tokens))

# We set up our decoder to return full output sequences,
# and to return internal states as well. We don't use the
# return states in the training model, but we will use them in inference.
decoder_lstm = keras.layers.LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state=encoder_states)
decoder_dense = keras.layers.Dense(num_decoder_tokens, activation="softmax")
decoder_outputs = decoder_dense(decoder_outputs)

# Define the model that will turn
# `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
model = keras.Model([encoder_inputs, decoder_inputs], decoder_outputs)

"""
## Train the model
"""

model.compile(
    optimizer="rmsprop", loss="categorical_crossentropy", metrics=["accuracy"]
)
model.fit(
    [encoder_input_data, decoder_input_data],
    decoder_target_data,
    batch_size=batch_size,
    epochs=epochs,
    validation_split=0.2,
)
# Save model
model.save("s2s_model.keras")

"""
## Run inference (sampling)

1. encode input and retrieve initial decoder state
2. run one step of decoder with this initial state
and a "start of sequence" token as target.
Output will be the next target token.
3. Repeat with the current target token and current states
"""

# Define sampling models
# Restore the model and construct the encoder and decoder.
model = keras.models.load_model("s2s_model.keras")

encoder_inputs = model.input[0]  # input_1
encoder_outputs, state_h_enc, state_c_enc = model.layers[2].output  # lstm_1
encoder_states = [state_h_enc, state_c_enc]
encoder_model = keras.Model(encoder_inputs, encoder_states)

decoder_inputs = model.input[1]  # input_2
decoder_state_input_h = keras.Input(shape=(latent_dim,))
decoder_state_input_c = keras.Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_lstm = model.layers[3]
decoder_outputs, state_h_dec, state_c_dec = decoder_lstm(
    decoder_inputs, initial_state=decoder_states_inputs
)
decoder_states = [state_h_dec, state_c_dec]
decoder_dense = model.layers[4]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = keras.Model(
    [decoder_inputs] + decoder_states_inputs, [decoder_outputs] + decoder_states
)

# Reverse-lookup token index to decode sequences back to
# something readable.
reverse_input_char_index = dict((i, char) for char, i in input_token_index.items())
reverse_target_char_index = dict((i, char) for char, i in target_token_index.items())


def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq, verbose=0)

    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0, target_token_index["\t"]] = 1.0

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ""
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict(
            [target_seq] + states_value, verbose=0
        )

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if sampled_char == "\n" or len(decoded_sentence) > max_decoder_seq_length:
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.0

        # Update states
        states_value = [h, c]
    return decoded_sentence


"""
You can now generate decoded sentences as such:
"""

for seq_index in range(20):
    # Take one sequence (part of the training set)
    # for trying out decoding.
    input_seq = encoder_input_data[seq_index : seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    print("-")
    print("Input sentence:", input_texts[seq_index])
    print("Decoded sentence:", decoded_sentence)

Number of samples: 5000
Number of unique input tokens: 67
Number of unique output tokens: 89
Max sequence length for inputs: 13
Max sequence length for outputs: 46
Epoch 1/50
32/32 ━━━━━━━━━━━━━━━━━━━━ 21s 556ms/step - accuracy: 0.6022 - loss: 2.3439 - val_accuracy: 0.6569 - val_loss: 1.6157
Epoch 2/50
32/32 ━━━━━━━━━━━━━━━━━━━━ 19s 519ms/step - accuracy: 0.6894 - loss: 1.3709 - val_accuracy: 0.6557 - val_loss: 1.5418
Epoch 3/50
32/32 ━━━━━━━━━━━━━━━━━━━━ 22s 570ms/step - accuracy: 0.6950 - loss: 1.2632 - val_accuracy: 0.6578 - val_loss: 1.3759
Epoch 4/50
32/32 ━━━━━━━━━━━━━━━━━━━━ 19s 521ms/step - accuracy: 0.6988 - loss: 1.1833 - val_accuracy: 0.6597 - val_loss: 1.2951
Epoch 5/50
32/32 ━━━━━━━━━━━━━━━━━━━━ 21s 535ms/step - accuracy: 0.6998 - loss: 1.1502 - val_accuracy: 0.6663 - val_loss: 1.2379
Epoch 6/50
32/32 ━━━━━━━━━━━━━━━━━━━━ 19s 588ms/step - accuracy: 0.7033 - loss: 1.1071 - val_accuracy: 0.6618 - val_loss: 1.2077
Epoch 7/50
32/32 ━━━━━━━━━━━━━━━━━━━━ 19s 565ms/step - accurac

50行目～53行目：インポート・セットアップ
63行目～66行目：データのダウンロード（URLから直接ができなかった）
72行目～77行目：構成
84行目～145行目：データの準備
152行目～172行目：モデルの構築
178行目～189行目：モデルの学習
203行目～265行目：推定の実行
272行目～：出力

##【問題2】イメージキャプショニングの学習済みモデルの実行

https://github.com/yunjey/pytorch-tutorial/tree/master/tutorials/03-advanced/image_captioning

In [1]:
# 必要なリポジトリをクローン
!git clone https://github.com/yunjey/pytorch-tutorial.git

# 該当ディレクトリに移動
%cd pytorch-tutorial/tutorials/03-advanced/image_captioning/

Cloning into 'pytorch-tutorial'...
remote: Enumerating objects: 917, done.
remote: Total 917 (delta 0), reused 0 (delta 0), pack-reused 917 (from 1)
Receiving objects: 100% (917/917), 12.80 MiB | 18.84 MiB/s, done.
Resolving deltas: 100% (491/491), done.
/content/pytorch-tutorial/tutorials/03-advanced/image_captioning


In [3]:
# 必要なフォルダを作成
!mkdir -p ./models
!mkdir -p ./data

# アップロードしたファイルを移動
!mv /encoder-5-3000.pkl ./models/
!mv /decoder-5-3000.pkl ./models/
!mv /vocab.pkl ./data/

In [4]:
!ls ./models/
!ls ./data/
!ls

decoder-5-3000.pkl  encoder-5-3000.pkl
vocab.pkl
build_vocab.py	data_loader.py	model.py  png	     requirements.txt  sample.py
data		download.sh	models	  README.md  resize.py	       train.py


In [5]:
!ls

build_vocab.py	data_loader.py	model.py  png	     requirements.txt  sample.py
data		download.sh	models	  README.md  resize.py	       train.py


In [6]:
!python sample.py --image='png/example.png'

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet152_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet152_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet152-394f9c45.pth" to /root/.cache/torch/hub/checkpoints/resnet152-394f9c45.pth
100% 230M/230M [00:02<00:00, 98.8MB/s]
/content/pytorch-tutorial/tutorials/03-advanced/image_captioning/sample.py:43: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pi

##【問題3】Kerasで動かしたい場合はどうするかを調査

PyTorchで実装したSequence to SequenceモデルをKerasに移植する際には、モデル構造の再現と学習済みの重みの移行が必要

---

## 1. **PyTorchモデルの理解と構造の再現**

Kerasでモデルを再現するためには、PyTorchのモデル構造を詳細に理解する必要があります。  
### 手順：
1. **PyTorchモデルの調査**  
   - 使用されているレイヤー（例: `LSTM`, `GRU`, `Embedding`, `Linear`など）の確認。
   - 入力サイズ、隠れ層のサイズ、出力サイズ、アクティベーション関数、ドロップアウト率などのハイパーパラメータを記録。

2. **Kerasでのモデル定義**  
   - PyTorchの各レイヤーに対応するKerasレイヤーを使用してモデルを再構築。  
     例:  
     - `torch.nn.Embedding` → `tf.keras.layers.Embedding`
     - `torch.nn.LSTM` → `tf.keras.layers.LSTM`
     - `torch.nn.Linear` → `tf.keras.layers.Dense`
   - 注意:  
     - KerasとPyTorchではレイヤーの順番や出力形式が異なる場合があるため、必要に応じて調整する。

---

## 2. **学習済みの重みの変換**

PyTorchの学習済み重みをKerasで利用するためには、重みを適切な形式で保存し、Kerasモデルに読み込む必要があります。  

### 手順：
1. **PyTorchの重みを保存**  
   - PyTorchモデルの状態辞書を取得し、保存します。
     ```python
     torch.save(model.state_dict(), 'model_weights.pth')
     ```

2. **重みの構造を確認**  
   - PyTorchの状態辞書は辞書型オブジェクトで、各レイヤー名とその重みが含まれています。
     ```python
     state_dict = torch.load('model_weights.pth')
     for key, value in state_dict.items():
         print(key, value.shape)
     ```

3. **Keras用に変換**  
   - PyTorchとKerasでは重みの形状が異なる場合があります（例: RNNの順序、Linear層の転置）。  
     必要に応じて重みを変換します。
     ```python
     import numpy as np
     for key, value in state_dict.items():
         state_dict[key] = value.numpy()
     ```

4. **Kerasモデルへの重みのロード**  
   - Kerasでモデルを定義した後、PyTorchから変換した重みをKerasにロードします。
     ```python
     keras_model = ...  # Kerasで定義したモデル
     keras_model.set_weights([
         state_dict['embedding.weight'],  # 例: Embeddingレイヤーの重み
         np.transpose(state_dict['lstm.weight_ih_l0']),  # 例: LSTMの入力重み
         np.transpose(state_dict['lstm.weight_hh_l0']),  # 例: LSTMの隠れ状態重み
         state_dict['lstm.bias_ih_l0'],  # 例: LSTMのバイアス
         state_dict['lstm.bias_hh_l0'],  # 例: LSTMのバイアス
         # その他のレイヤーも対応する順序で設定
     ])
     ```

---

## 3. **動作確認と微調整**

Kerasでモデルを動かす前に、PyTorchでの動作を再現できているかを確認します。  

1. **テストデータでの出力確認**  
   - 同じ入力を使用して、PyTorchモデルとKerasモデルの出力が一致するか確認します。

2. **誤差が出る場合の対応**  
   - 異なる出力が得られる場合、以下の点を確認します:
     - 重みの順序や形状が正しいか。
     - Kerasの初期状態やオプションがPyTorchと一致しているか。
     - 損失関数やオプティマイザの設定が一致しているか。

---

## 補足: KerasとPyTorchの主要な違い

1. **バッチの扱い**  
   - PyTorch: `(batch_size, seq_length, feature_dim)`  
   - Keras: `(batch_size, feature_dim, seq_length)`やその逆の場合がある。

2. **重みの形式**  
   - RNN系レイヤー（LSTM, GRUなど）では、PyTorchとKerasで順序や形式が異なる場合が多い。

3. **データ型**  
   - PyTorchはデフォルトで`float32`を使用するが、Kerasではモデル定義時に精度を指定可能。

---

## まとめ

1. **モデル構造をKerasで再現**  
   PyTorchのレイヤーに対応するKerasレイヤーを選び、モデルを構築します。

2. **重みを変換して適用**  
   PyTorchの重みを保存し、適切な形に変換した上でKerasモデルにロードします。

3. **動作確認と微調整**  
   入力を用いてPyTorchとKerasで同一の出力を得られるように確認します。

これらの手順を踏むことで、PyTorchの学習済みモデルをKerasで活用できるようになります。